In [ ]:
import numpy as np

class CalLoss:
    def __init__(self, y, y_pred):
        self.y = y
        self.y_predicted = y_pred
        if y.shape != y_pred.shape:
            raise ValueError(f"Shape mismatch: y shape is {self.y.shape}, y_predicted shape is {self.y_predicted.shape}")

class CrossEntropy(CalLoss):
    def give_celoss(self):
        epsilon = 1e-8  # Small value to prevent log(0)
        return -np.mean(self.y * np.log(self.y_predicted + epsilon))

    def Give_cegrad(self):
        epsilon = 1e-8  # Prevent division by zero
        grad = -self.y / (self.y_predicted + epsilon)
        return grad

class SquaredError(CalLoss):
    def give_seloss(self):
        return np.mean((self.y - self.y_predicted) ** 2)

    def Give_segrad(self):
        grad = -2 * (self.y - self.y_predicted)
        return np.clip(grad, -1, 1)  # Clip to prevent exploding gradients

class callloss(CalLoss):
    def __init__(self, loss_function, y, y_pred):
        self.loss_function = loss_function.lower()
        super().__init__(y, y_pred)

    def give_loss(self):
        if self.loss_function == 'ce':
            return CrossEntropy(self.y, self.y_predicted).give_celoss()
        elif self.loss_function == 'se':
            return SquaredError(self.y, self.y_predicted).give_seloss()
        else:
            raise ValueError(f"Unknown loss function: {self.loss_function}")

    def give_gradloss(self):
        if self.loss_function == 'ce':
            return CrossEntropy(self.y, self.y_predicted).Give_cegrad()
        elif self.loss_function == 'se':
            return SquaredError(self.y, self.y_predicted).Give_segrad()
        else:
            raise ValueError(f"Unknown loss function: {self.loss_function}")
